In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import trange
import os
import json
import pandas as pd
from banditpy.models.rnn_models import BanditTrainer2Arm


bt = BanditTrainer2Arm(n_train_sessions=2000, n_test_sessions=200)
bt.train(structured=True)
bt.save_model()

Starting training for 2000 sessions...
Training Session 100/2000, Avg Loss (last 100): 0.7284
Training Session 200/2000, Avg Loss (last 100): 0.3021
Training Session 300/2000, Avg Loss (last 100): 0.1896
Training Session 400/2000, Avg Loss (last 100): 0.3096
Training Session 500/2000, Avg Loss (last 100): 0.1144
Training Session 600/2000, Avg Loss (last 100): 0.2204
Training Session 700/2000, Avg Loss (last 100): 0.1028
Training Session 800/2000, Avg Loss (last 100): 0.1319
Training Session 900/2000, Avg Loss (last 100): 0.1765
Training Session 1000/2000, Avg Loss (last 100): 0.1744
Training Session 1100/2000, Avg Loss (last 100): 0.0379
Training Session 1200/2000, Avg Loss (last 100): 0.1641
Training Session 1300/2000, Avg Loss (last 100): 0.0798
Training Session 1400/2000, Avg Loss (last 100): 0.1263
Training Session 1500/2000, Avg Loss (last 100): 0.0998
Training Session 1600/2000, Avg Loss (last 100): 0.1006
Training Session 1700/2000, Avg Loss (last 100): 0.0799
Training Session 1

In [10]:
from banditpy.core import Bandit2Arm
import matplotlib.pyplot as plt


df = bt.evaluate(structured=True)
task = Bandit2Arm(
    probs=df.loc[:, ["arm1_reward_prob", "arm2_reward_prob"]].to_numpy(),
    choices=df["chosen_action"].to_numpy(),
    rewards=df["reward"].to_numpy(),
    session_ids=df["session_id"].to_numpy(),
)
plt.plot(task.get_performance())

Starting evaluation with fixed weights...
Model loaded from two_arm_task_model.pt
Evaluation Session 50/200 complete.
Evaluation Session 100/200 complete.
Evaluation Session 150/200 complete.
Evaluation Session 200/200 complete.
Evaluation complete.


In [12]:
df.chosen_action.unique()

array([2])

In [9]:
import matplotlib.pyplot as plt

plt.plot(bt.training_loss_history)

In [73]:
ht = bt.get_model_weights()["rnn.weight_hh_l0"]

In [74]:
plt.pcolormesh(ht)